# Discontinuous Galerkin for the Wave Equation

We solve the first order wave equation by a matrix-free explicit DG method:

\begin{eqnarray*}
\frac{\partial p}{\partial t} & = & \operatorname{div} u \\
\frac{\partial u}{\partial t} & = & \nabla p
\end{eqnarray*}


Using DG discretization in space we obtain the ODE system
\begin{eqnarray*}
M_p \dot{p} & = & -B^T u \\
M_u \dot{u} & = & B p,
\end{eqnarray*}

with mass-matrices $M_p$ and $M_u$, and the discretization matrix $B$ for the gradient, and $-B^T$ for the divergence. 


The DG bilinear-form for the gradient is:

$$
b(p,v) = \sum_{T}
\Big\{ \int_T \nabla p  \, v + \int_{\partial T} (\{ p \} - p) \, v_n \, ds \Big\},
$$
where $\{ p \}$ is the average of $p$ on the facet.

The computational advantages of a proper version of DG is:

* universal element-matrices for the differntial operator $B$
* cheaply invertible mass matrices (orthogonal polynomials, sum-factorization)


In [1]:
from ngsolve import *
from netgen.occ import *
from time import time

from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo

box = Box((-1,-1,-1), (1,1,0))
sp = Sphere((0.5,0,0), 0.2)
shape = box-sp
geo = OCCGeometry(shape)

h = 0.1
        
mesh = Mesh( geo.GenerateMesh(maxh=h))
mesh.Curve(3)
Draw(mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

In [2]:
order = 4
fes_p = L2(mesh, order=order, all_dofs_together=True)
fes_u = VectorL2(mesh, order=order, piola=True)
fes_tr = FacetFESpace(mesh, order=order)

print ("ndof_p = ", fes_p.ndof, "+", fes_tr.ndof, ", ndof_u =", fes_u.ndof)

traceop = fes_p.TraceOperator(fes_tr, average=True) 

gfu = GridFunction(fes_u)
gfp = GridFunction(fes_p)
gftr = GridFunction(fes_tr)

gfp.Interpolate( exp(-400*(x**2+y**2+z**2)))
gftr.vec.data = traceop * gfp.vec

ndof_p =  650405 + 585960 , ndof_u = 1951215


In [3]:
p = fes_p.TrialFunction()
v = fes_u.TestFunction()
phat = fes_tr.TrialFunction()

n = specialcf.normal(mesh.dim)

$$
b(p,v) = \sum_{T}
\Big\{ \int_T \nabla p  \, v + \int_{\partial T} (\{ p \} - p) \, v_n \, ds \Big\},
$$

where $\{ p \}$ is the average of $p$ on the facet.


In [4]:
Bel = BilinearForm(trialspace=fes_p, testspace=fes_u, geom_free = True)
Bel += grad(p)*v * dx -p*(v*n) * dx(element_boundary=True)
Bel.Assemble()

Btr = BilinearForm(trialspace=fes_tr, testspace=fes_u, geom_free = True)
Btr += phat * (v*n) *dx(element_boundary=True)
Btr.Assemble();

B = Bel.mat + Btr.mat @ traceop

In [5]:
invmassp = fes_p.Mass(1).Inverse()
invmassu = fes_u.Mass(1).Inverse()

In [6]:
gfp.Interpolate( exp(-100*(x**2+y**2+z**2)))
gfu.vec[:] = 0

scene = Draw (gftr, draw_vol=False, order=3);

t = 0
dt = 0.3 * h / (order+1)**2 
tend = 1.2

op1 = dt * invmassu @ B
op2 = dt * invmassp @ B.T

cnt = 0
with TaskManager(): 
    while t < tend:
        t = t+dt
        gfu.vec.data += op1 * gfp.vec
        gfp.vec.data -= op2 * gfu.vec
        cnt = cnt+1
        if cnt%10 == 0:
            gftr.vec.data = traceop * gfp.vec
            scene.Redraw()

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

KeyboardInterrupt: 

## Time-stepping on the device

In [7]:
try:
    import ngsolve.ngscuda
except:
    print ("Sorry, no cuda device")

Sorry, no cuda device


In [8]:
gfp.Interpolate( exp(-100*(x**2+y**2+z**2)))
gfu.vec[:] = 0

scene = Draw (gftr, draw_vol=False, order=3);

t = 0
dt = 0.5 * h / (order+1)**2 / 2
tend = 0.1

op1 = (dt * invmassu @ B).CreateDeviceMatrix()
op2 = (dt * invmassp @ B.T).CreateDeviceMatrix()

devu = gfu.vec.CreateDeviceVector(copy=True)
devp = gfp.vec.CreateDeviceVector(copy=True)

cnt = 0
with TaskManager(): 
    while t < tend:
        t = t+dt
        devu += op1 * devp
        devp -= op2 * devu
        cnt = cnt+1
        if cnt%10 == 0:
            gfp.vec.data = devp
            gftr.vec.data = traceop * gfp.vec
            scene.Redraw()

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

In [9]:
print (op1.GetOperatorInfo())

ProductMatrix, h = 1951215, w = 650405
  ScaleMatrix, scale = 0.001, h = 1951215, w = 1951215
    SumMatrix, h = 1951215, w = 1951215
      SumMatrix, h = 1951215, w = 1951215
        BlockDiagonalMatrixSoA (bs = 3x3), h = 1951215, w = 1951215
        ProductMatrix, h = 1951215, w = 1951215
          ProductMatrix, h = 1951215, w = 18000
            Transpose, h = 1951215, w = 18000
              ConstantEBEMatrix (bs = 125x35), h = 18000, w = 1951215
            BlockDiagonalMatrixSoA (bs = 3x3), h = 18000, w = 18000
          ConstantEBEMatrix (bs = 125x35), h = 18000, w = 1951215
      ProductMatrix, h = 1951215, w = 1951215
        ProductMatrix, h = 1951215, w = 24000
          Transpose, h = 1951215, w = 24000
            ConstantEBEMatrix (bs = 125x35), h = 24000, w = 1951215
          BlockDiagonalMatrixSoA (bs = 3x3), h = 24000, w = 24000
        ConstantEBEMatrix (bs = 125x35), h = 24000, w = 1951215
  SumMatrix, h = 1951215, w = 650405
    SumMatrix, h = 1951215, w = 650405


In [10]:
ts = time()
steps = 10
for i in range(steps):
    devu += op1 * devp
    devp -= op2 * devu
te = time()
print ("ndof = ", gfp.space.ndof, "+", gfu.space.ndof, ", time per step =", (te-ts)/steps)

ndof =  650405 + 1951215 , time per step = 0.0533890962600708


On the A-100:

## Time-domain PML

PML (perfectly matched layers) is a method for approximating outgoing waves on a truncated domain. Its time domain version leads to additional field variables in the PML region, which are coupled via time-derivatives only.

Formulation of B. Kapidani, J. Schöberl, [arxiv](https://arxiv.org/abs/2002.08733)

In [11]:
from ring_resonator_import import *
from ngsolve import *
from ngsolve.webgui import Draw

In [12]:
fes_u = VectorL2(mesh, order=order, piola=True, order_policy=ORDER_POLICY.CONSTANT)
fes_p = L2(mesh, order=order+1, all_dofs_together=True, order_policy=ORDER_POLICY.CONSTANT)
fes_tr = FacetFESpace(mesh, order=order+1)
fes_hdiv = HDiv(mesh, order=order+1, orderinner=1)

n = specialcf.normal(2) 

p,q = fes_p.TnT()
u,v = fes_u.TnT()
ptr,qtr = fes_tr.TnT()
uhdiv,vhdiv = fes_hdiv.TnT()

Bel = BilinearForm(grad(p)*v*dx - p*(v*n)*dx(element_boundary=True), geom_free=True).Assemble()
Btr = BilinearForm(0.5*ptr*(n*v)*dx(element_boundary=True), geom_free=True).Assemble()
Bstab = BilinearForm(p*(vhdiv*n)*dx(element_boundary=True),  geom_free=True).Assemble()


nvec = { mat : ((normals[mat][0], normals[mat][1]) if mat in normals else (0,0)) for mat in mesh.GetMaterials() }

cfn = CF( [CF(nvec[mat]) for mat in mesh.GetMaterials()])
cft = CF( ( cfn[1], -cfn[0] ) )

pml1d = mesh.Materials("pml_default.*|pml_normal.*")
eps = CF([eps_r[mat] for mat in mesh.GetMaterials()])


fes = fes_p*fes_p*fes_u*fes_u*fes_hdiv
emb_p, emb_phat, emb_u, emb_uhat, emb_hdiv = fes.embeddings

# gradient operator
traceop = fes_p.TraceOperator(fes_tr, False)
fullB = emb_u @ (Bel.mat + Btr.mat @ traceop) @ emb_p.T + emb_hdiv@Bstab.mat@emb_p.T

# mass matrices
invmassp = fes_p.Mass(eps).Inverse()
invmassu = fes_u.Mass(Id(mesh.dim)).Inverse()
Mstab = BilinearForm(uhdiv*vhdiv*dx, diagonal=True).Assemble()
Mstabinv = Mstab.mat.Inverse()


invp = emb_p @ invmassp @ emb_p.T + emb_phat @ invmassp @ emb_phat.T
invu = emb_u @ invmassu @ emb_u.T + emb_uhat @ invmassu @ emb_uhat.T + emb_hdiv@Mstabinv@emb_hdiv.T


# damping matrices
dampp1 = fes_p.Mass (eps, definedon=pml1d)
dampp2 = fes_p.Mass (eps, definedon=mesh.Materials("pml_corner"))
dampu1 = fes_u.Mass (OuterProduct(cfn,cfn), definedon=pml1d)
dampu2 = fes_u.Mass (OuterProduct(cft,cft), definedon=pml1d)

dampingu = emb_u @ dampu1 @ emb_u.T + (-emb_u + emb_uhat) @ dampu2 @ (emb_u.T + emb_uhat.T)
dampingp = emb_p @ dampp1 @ emb_p.T + emb_p @ dampp2 @ (2*emb_p.T-emb_phat.T) + emb_phat @ dampp2 @ emb_p.T


# source term
Lsrc  = LinearForm(gfsource*q*dx(element_boundary=True)).Assemble()
srcvec = emb_p * (invmassp*Lsrc.vec).Evaluate()

# time-envelope:
def Envelope(t):
    if abs((t-tpeak)/tpeak) < 1:
        return (2*exp(1)/sqrt(pi))*sin(2*pi*fcen*t)*exp (-1/(1-((t-tpeak)/tpeak)**2))
    else:
        return 0

In [ ]:
gfu = GridFunction(fes)
scene = Draw (gfu.components[0], order=3, min=-0.05, max=0.05, autoscale=False)

t = 0
tend = 15
tau = 2e-4
i = 0
sigma = 10   # pml damping parameter

op1 = invp@(-fullB.T-sigma*dampingp) 
op2 = invu@(fullB-sigma*dampingu)
with TaskManager(): 
    while t < tend:

        gfu.vec.data += tau*Envelope(t)*srcvec
        gfu.vec.data += tau*op1*gfu.vec
        gfu.vec.data += tau*op2*gfu.vec        

        t += tau
        i += 1
        if i%20 == 0:
            scene.Redraw()

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

The differential operators and coupling terms to the pml - variables are represented via linear operators;

In [ ]:
print (fullB.GetOperatorInfo())
print (dampingp.GetOperatorInfo())

In [ ]:
print (op1.GetOperatorInfo())